In [3]:
from  tiny.package import *
from  tiny.util import *
from  tiny.lda import *
from  tiny.tfidf import *
print('begin')
# cntTf = get_cntTf('app')

tmp=extend_package(version=1, mini=False,type='combine_type')
tmp.head().sum(axis=0)

2018-09-16 13:13:28,369 util_log.py[33] INFO Begin to run extend_package with:[], {'version': 1, 'mini': False, 'type': 'combine_type'}
2018-09-16 13:13:28,374 util_cache_file.py[69] DEBUG fn:extend_package, para:[], kw:{'version': 1, 'mini': False, 'type': 'combine_type'}
2018-09-16 13:13:28,377 util_cache_file.py[20] DEBUG try to read cache from file:./cache/extend_package_[]_{'version': 1, 'mini': False, 'type': 'combine_type'}.pkl, type:<class 'type'>
2018-09-16 13:13:28,468 util_cache_file.py[25] DEBUG Load to <class 'pandas.core.sparse.frame.SparseDataFrame'> for pkl@./cache/extend_package_[]_{'version': 1, 'mini': False, 'type': 'combine_type'}.pkl
2018-09-16 13:13:28,471 util_cache_file.py[31] DEBUG Return 72727 resut from file cache:./cache/extend_package_[]_{'version': 1, 'mini': False, 'type': 'combine_type'}.pkl
2018-09-16 13:13:28,474 util_log.py[38] INFO cost: 0.11 sec: ==='extend_package' end ([], {'version': 1, 'mini': False, 'type': 'combine_type'}) 


begin


count_combine_type_ACT(动作类游戏)_动作冒险                  0.000000
count_combine_type_ACT(动作类游戏)_格斗                    0.000000
count_combine_type_RPG(角色扮演游戏)_ARPG(即时战斗角色扮演游戏)     0.000000
count_combine_type_RPG(角色扮演游戏)_MOBA(多人在线战术竞技游戏)     0.000000
count_combine_type_RPG(角色扮演游戏)_回合制                  0.000000
count_combine_type_RPG(角色扮演游戏)_放置类                  0.000000
count_combine_type_SIM(模拟游戏)_恋爱养成                   0.000000
count_combine_type_SIM(模拟游戏)_模拟养成                   0.000000
count_combine_type_SIM(模拟游戏)_模拟经营                   0.000000
count_combine_type_SIM(模拟游戏)_沙盒                     0.000000
count_combine_type_SLG(策略游戏)_塔防                     0.000000
count_combine_type_SLG(策略游戏)_策略                     0.000000
count_combine_type_TAB(桌面游戏)_桌游                     0.000000
count_combine_type_TAB(桌面游戏)_棋牌                    32.000000
count_combine_type_Unknown                         77.000000
count_combine_type_交通出行_互联网巴士                       0.000000
count_combine_type_交通出行_

In [5]:
print(type(tmp))
print(tmp.density)

<class 'pandas.core.sparse.frame.SparseDataFrame'>
0.012623831941445904


In [ ]:
for col in tmp.columns:
    if 'Unknown' in col:
        print(col)
 


In [ ]:
pkg_label = get_package_label()
pkg_label.set_index('package', inplace=True)
pkg_label

In [ ]:
extend_time_span(version=0, trunc_long_time=False, mini=True, groupby=['device'], prefix='tol')


In [ ]:
columns = [ col for col in install.columns ]
print(columns[:4])

In [ ]:
vectorizer = CountVectorizer()
cntTf_app = vectorizer.fit_transform(apps)

str(cntTf_app[:3])
# cntTf_app = pd.DataFrame(cntTf_app,
#                          columns=vectorizer.get_feature_names(),
#                          index=deviceid_packages.device)

In [ ]:
mini = get_start_closed()
tmp[tmp.device.isin(mini.device)][['device','duration_max_min', 'tol_day_cnt_nunique', 'gap']].sort_values('gap', ascending=False)


#f993126b72f6297440c2dc8534b20a0b  49
#fbf985012f537449b455d6a554479f8b  1161




In [ ]:
from  tiny.lda import *
get_lda_from_usage()

In [ ]:
device= 'fa68862f9484c62aca5c3a4b8a2c70bc'
mini[mini.device==device].sort_values('start')

In [ ]:
tmp[tmp.device==device]

In [ ]:
df2

In [ ]:
#df2.set_index('device', inplace=True)
df.sort_index(ascending=False)


In [ ]:
split.head()

In [ ]:
print(pd.to_datetime('1488332491588', unit='ms'))  #fc6383d2bdb463fd01db6731594a6d36	07e967d75aab2f6a52c558695a572a7c
print(pd.to_datetime('1488332506604', unit='ms'))
print(pd.to_datetime('1488363700875', unit='ms'))
print(pd.to_datetime('1488363707487', unit='ms'))

In [ ]:
tmp = extend_time(tmp,2)
#tmp['start'].astype
#tmp['start'] = pd.to_datetime(tmp['start'])
#tmp['start'].dt.date

In [ ]:

# def extend_percent(df):
#     total = get_percent_duration(df, ['device'], 'total')

#     max_week = get_max_week(df)

#     merge = df.merge(max_week, on=['device', 'weekbegin'])

#     max = get_percent_duration(merge, ['device'], prefix='max')

#     return pd.concat( [total, max], axis=1 )


# total = get_percent_duration(df, ['device'], 'total')


# df = tmp
# max_week = get_max_week(df)
# max_week.head()
# merge = df.merge(max_week, on=['device', 'weekbegin'])

 
# def get_percent_duration(df, groupby=['device', 'weekday'], prefix=None):
#     prefix = groupby[-1] if prefix is None else prefix
#     columns = [key for key in df.columns if 'span_' in key]
#     gp_map = [(key, 'sum') for key in columns if 'span_' in key]
#     gp_map = dict(gp_map)
#     # print(gp_map)

#     gp_map['package'] = 'nunique'
#     gp_map['start_base'] = 'nunique'
#     df = df.groupby(groupby).agg(gp_map)
#     #     df = df.groupby(groupby).agg({'span_0':'sum','span_1':'sum','span_2':'sum',
#     #                                                  'span_3':'sum','package':'nunique' })
#     df['total'] = df[[key for key in columns if 'span_' in key]].sum(axis=1)

#     for col in columns:
#         df[f'{col}_p'] = round(df[col] / df['total'], 3)

#     df.rename({'package':'package_cnt', 'start_base':'start_base_cnt'}, axis=1, inplace=True)

#     df.columns = [f'{prefix}_{key}' for key in df.columns]
#     return df

per = extend_percent(tmp)
per.head()

# max_week = get_max_week(tmp)
# max_week

# merge= tmp.merge(max_week, on = ['device','weekbegin'])

# print(len(merge))

# get_percent_duration(merge, ['device'], prefix='max')


    

In [ ]:
per
#tmp[tmp.device=='fc6383d2bdb463fd01db6731594a6d36'].sort_values('start')

In [ ]:
print('yes')

In [ ]:
#tmp['start_base']  = tmp['start'].dt.date
# tmp = tmp[tmp.device.isin(['fff885dafce62310951c835247f30a38',
# 'fc65f1731b785a8ce82d20a30dee9259'])]

df = tmp.groupby(['device','weekbegin']).package.nunique().to_frame().reset_index()

# .to_frame().reset_index()
# df.sort_values(['device', 'package'])

df.sort_values(by=['device', 'package', 'weekbegin'], ascending=False).groupby('device').nth(0)


In [ ]:
xx = df



In [ ]:
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from xgboost.sklearn import XGBRegressor
import lightgbm as lgb
from sklearn.cross_validation import train_test_split

from tiny.lda import *
from  tiny.util import *

# New add
# deviceid_train.rename({'device_id':'device'}, axis=1, inplace=True)
deviceid_train = get_lda_feature()
deviceid_train2 = get_lda_from_usage(mini=mini)

core_list = ['0', '1', '2', '3','4']
# for col in core_list:
#     deviceid_train_2[col] =  deviceid_train_2[col].apply(lambda val: 1 if val > 0 else 0)
deviceid_train = pd.concat([deviceid_train,deviceid_train2[core_list] ], axis=1)

deviceid_train = extend_feature(span_no=24, input=deviceid_train, trunc_long_time=False)

#deviceid_train = extend_feature(span_no=4,input=deviceid_train,  trunc_long_time=False)



#print(len(deviceid_train))
#deviceid_train.groupby('max_day_cnt')['max_day_cnt'].count()

#
#
# col_drop = [item for item in deviceid_train.columns if 'max_' in str(item)]
# deviceid_train.drop(columns=col_drop, inplace=True )

#deviceid_train.drop(columns=['tfidf_sum'], inplace=True )
deviceid_train.head()


train=deviceid_train[deviceid_train['sex'].notnull()]
test=deviceid_train[deviceid_train['sex'].isnull()]

X = train.drop(['sex', 'age', 'sex_age', 'device'], axis=1)
Y = train['sex_age']
Y_CAT = pd.Categorical(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y_CAT.labels, test_size=0.3, random_state=666)
# lgb_train = lgb.Dataset(X_train, label=y_train)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'max_depth': 3,
    'metric': {'multi_logloss'},
    'num_class': 22,
    'objective': 'multiclass',
    'random_state': 47,

    "min_data_in_leaf":1000,
    'verbose': -1

}


gbm = LGBMClassifier(
        n_estimators=1000,

        )

gbm.set_params(**params)

print(gbm)

gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)],early_stopping_rounds=100,)

#y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)#
print('Feature importances:', list(gbm.feature_importances_))

best = round(gbm.best_score_.get('valid_0').get('multi_logloss'), 5)
best


pre_x=test.drop(['sex','age','sex_age','device'],axis=1)


In [ ]:
sub=pd.DataFrame(gbm.predict_proba(pre_x.values,num_iteration=gbm.best_iteration_))

sub.head()

In [ ]:
tmp = extend_cols(tmp)
tmp, tmp_new = split_days(tmp)

tmp.head()

In [ ]:
tmp, tmp_new = split_days(tmp)
tmp.head()

In [ ]:
trainn_label = get_train_label()
trainn_label.head()

In [ ]:
from  tiny.util import *

tmp = extend_feature( span_no=2, trunc_long_time=False, mini=True)

In [ ]:
tmp.head()

In [ ]:
tmp = extend_time_span(version=1, trunc_long_time=False, mini=True, groupby=['device'], prefix='tol')
tmp.head()

In [ ]:
from tiny.lda import *
from  tiny.util import *

tmp = extend_package(version=1)
tmp.head(3).sum(axis=1)

In [ ]:
tmp = tmp/tmp

tmp.head(3).sum(axis=1)

In [ ]:
tmp.head(3).count(axis=1)

In [ ]:
tmp.head(3)